In [1]:
import shutil
from pathlib import Path

# Define input paths
datasets = {
    "human_train": (Path("/kaggle/input/face-detection-dataset/images/train"),
                    Path("/kaggle/input/face-detection-dataset/labels/train"), 0),
    "human_val": (Path("/kaggle/input/face-detection-dataset/images/val"),
                  Path("/kaggle/input/face-detection-dataset/labels/val"), 0),
    "anime_train": (Path("/kaggle/input/annotated-anime-faces-dataset/train/images"),
                    Path("/kaggle/input/annotated-anime-faces-dataset/train/labels"), 1),
    "anime_val": (Path("/kaggle/input/annotated-anime-faces-dataset/valid/images"),
                  Path("/kaggle/input/annotated-anime-faces-dataset/valid/labels"), 1),
}

# Define output paths
output_dirs = {
    "train": ("combined_dataset/images/train", "combined_dataset/labels/train"),
    "val": ("combined_dataset/images/val", "combined_dataset/labels/val"),
}

# Create output directories
for dirs in output_dirs.values():
    Path(dirs[0]).mkdir(parents=True, exist_ok=True)
    Path(dirs[1]).mkdir(parents=True, exist_ok=True)

def copy_and_relabel(source_images, source_labels, output_images, output_labels, new_class):
    for img_path in source_images.glob("*.jpg"):
        shutil.copy(img_path, Path(output_images) / img_path.name)

    for label_path in source_labels.glob("*.txt"):
        try:
            with open(label_path, 'r') as f:
                lines = f.readlines()
            with open(Path(output_labels) / label_path.name, 'w') as out:
                for line in lines:
                    parts = line.strip().split()
                    parts[0] = str(new_class)  # Update the class
                    out.write(' '.join(parts) + '\n')
        except Exception as e:
            print(f"Error processing {label_path}: {e}")

# Copy datasets
copy_and_relabel(datasets["human_train"][0], datasets["human_train"][1], output_dirs["train"][0], output_dirs["train"][1], datasets["human_train"][2])
copy_and_relabel(datasets["human_val"][0], datasets["human_val"][1], output_dirs["val"][0], output_dirs["val"][1], datasets["human_val"][2])
copy_and_relabel(datasets["anime_train"][0], datasets["anime_train"][1], output_dirs["train"][0], output_dirs["train"][1], datasets["anime_train"][2])
copy_and_relabel(datasets["anime_val"][0], datasets["anime_val"][1], output_dirs["val"][0], output_dirs["val"][1], datasets["anime_val"][2])

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.3/886.3 kB 20.9 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import yaml

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
cfg = {
    "path": "../combined_dataset",  # Base path for your dataset
    "train": "images/train",        # Path to training images relative to `path`
    "val": "images/val",            # Path to validation images relative to `path`
    "names": {
        0: "Human",                 # Class 0: Human
        1: "Not Human"              # Class 1: Not Human
    },
    "nms": {
        "iou_thres": 0.3
    }
}

with open('data_config.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

In [5]:
import logging

logging.getLogger('ultralytics').setLevel(logging.WARNING)  # Only show warnings and errors

model = YOLO('yolo11n.pt')
model.train(data='data_config.yaml', epochs=20, opset=9, verbose=False, overlap_mask=True, imgsz=256)

100%|██████████| 5.35M/5.35M [00:00<00:00, 93.5MB/s]
100%|██████████| 755k/755k [00:00<00:00, 20.1MB/s]
2024-11-15 07:03:24,784	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-15 07:03:26,071	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
train: Scanning /kaggle/working/combined_dataset/labels/train... 18679 images, 385 backgrounds, 0 corrupt: 100%|██████████| 18679/18679 [00:16<00:00, 1118.73it/s]
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/opt/cond

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ce822d3e320>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [6]:
model.export(format='onnx')


'runs/detect/train/weights/best.onnx'

In [7]:
import shutil
from pathlib import Path

combined_dataset_path = Path("combined_dataset")

if combined_dataset_path.exists() and combined_dataset_path.is_dir():
    shutil.rmtree(combined_dataset_path)
    print(f"{combined_dataset_path} has been removed successfully.")
else:
    print(f"{combined_dataset_path} does not exist.")

combined_dataset has been removed successfully.
